# Machine Learning Stock Market Trader

This will serve as a guide to the program, updating everytime substantial progress has been made

Current Sprint:
    1- Simply visualize (line graph) market data
    2- Add other simple statistics such as 5-day MA, 30-day MA, ect.
    3- Add advanced indicators
Next Sprint:
    - Formulate simple Linear Regression Model
        -> Optimize the hell out of it
